In [ ]:
# Install dependencies directly in the notebook kernel
import sys
!{sys.executable} -m pip install -r requirements.txt

# Visualize Model Predictions
This notebook allows you to visualize 3D lung scans and see what your trained model predicts for them.
**Note:** You can run this locally if you have downloaded `best_model.pth` and a few `.npy` sample files.

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import os

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Import model after checking device to isolate issues
try:
    from model import LungCancer3DCNN
    print("Successfully imported LungCancer3DCNN from model.py")
except ImportError as e:
    print(f"Error importing model: {e}")
    print("Make sure 'model.py' is in the same directory as this notebook.")

Using device: cpu
Successfully imported LungCancer3DCNN from model.py


In [2]:
# Load the model
model = LungCancer3DCNN().to(device)
model_path = 'best_model.pth'

if os.path.exists(model_path):
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    print(f"✓ Model loaded from {model_path}")
    print(f"  Best Validation Accuracy: {checkpoint.get('val_accuracy', 0):.4f}")
else:
    print(f"Error: {model_path} not found. Please download it first.")

✓ Model loaded from best_model.pth
  Best Validation Accuracy: 0.9338


In [ ]:
import zipfile
import io

# --- CONFIGURATION ---
# Path to your local zip file
zip_file_path = "luna16_processed.zip" 
# ---------------------

def load_from_zip_and_predict(zip_path):
    if not os.path.exists(zip_path):
        print(f"Error: Zip file '{zip_path}' not found. Please make sure it is in the same folder.")
        return

    print(f"Opening {zip_path}...")
    
    samples_found = []

    try:
        with zipfile.ZipFile(zip_path, 'r') as outer_zip:
            # List files in outer zip
            outer_files = outer_zip.namelist()
            
            # Look for inner zip files (subsets)
            subset_zips = [f for f in outer_files if f.endswith('.zip')]
            
            if not subset_zips:
                print("No inner zip files found. Checking for direct .npy files...")
                # Fallback: Check directly in outer zip
                pos = next((f for f in outer_files if "_pos_" in f and f.endswith('.npy')), None)
                neg = next((f for f in outer_files if "_neg_" in f and f.endswith('.npy')), None)
                if pos: samples_found.append(("root", pos, outer_zip.read(pos)))
                if neg: samples_found.append(("root", neg, outer_zip.read(neg)))
            else:
                # Search inside nested zips
                for inner_zip_name in subset_zips:
                    if len(samples_found) >= 2: break # Stop if we have enough samples
                    
                    print(f"Checking inside {inner_zip_name}...")
                    try:
                        # Read inner zip into memory
                        inner_zip_data = outer_zip.read(inner_zip_name)
                        with zipfile.ZipFile(io.BytesIO(inner_zip_data)) as inner_zip:
                            inner_files = inner_zip.namelist()
                            
                            # Find pos and neg samples
                            # We check if we already have a pos/neg to avoid duplicates if iterating multiple subsets
                            has_pos = any("_pos_" in s[1] for s in samples_found)
                            has_neg = any("_neg_" in s[1] for s in samples_found)
                            
                            if not has_pos:
                                pos = next((f for f in inner_files if "_pos_" in f and f.endswith('.npy')), None)
                                if pos: samples_found.append((inner_zip_name, pos, inner_zip.read(pos)))
                            
                            if not has_neg:
                                neg = next((f for f in inner_files if "_neg_" in f and f.endswith('.npy')), None)
                                if neg: samples_found.append((inner_zip_name, neg, inner_zip.read(neg)))
                    except zipfile.BadZipFile:
                        print(f"  Skipping {inner_zip_name} (invalid zip)")
                        continue

        if not samples_found:
            print("No .npy files with '_pos_' or '_neg_' found in the zip file (checked nested zips too).")
            return

        print(f"Found {len(samples_found)} samples to test.")

        for zip_origin, file_name, file_content in samples_found:
            print(f"\nProcessing: {file_name} (from {zip_origin})")
            
            # Load numpy array from bytes
            try:
                data = np.load(io.BytesIO(file_content))
            except Exception as e:
                print(f"Error loading numpy array: {e}")
                continue
            
            print(f"Loaded shape: {data.shape}")
            
            # Visualize middle slice
            mid_slice = data.shape[0] // 2
            plt.figure(figsize=(5, 5))
            plt.imshow(data[mid_slice], cmap='gray')
            plt.title(f"File: {os.path.basename(file_name)}\nMiddle Slice (Z={mid_slice})")
            plt.axis('off')
            plt.show()
            
            # Prepare for model
            input_tensor = torch.from_numpy(data).float().unsqueeze(0).unsqueeze(0).to(device)
            
            # Run inference
            with torch.no_grad():
                output = model(input_tensor)
                probability = torch.sigmoid(output).item()
                
            print(f"Model Prediction Score: {probability:.4f}")
            print(f"Interpretation: {'CANCER (Positive)' if probability > 0.5 else 'NORMAL (Negative)'}")
            print("-" * 50)

    except Exception as e:
        print(f"An error occurred while processing the zip file: {e}")

# Run the function
load_from_zip_and_predict(zip_file_path)

Opening luna16_processed.zip...
No .npy files with '_pos_' or '_neg_' found in the zip file.
